In [2]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
import pandas as pd

In [ ]:
# 下载数据
ds = load_dataset("YBXL/JAMA-reasoning-instruction", cache_dir='/home/gy237/project/download_data')

In [27]:
print(ds['train'][0])

{'entries': [{'content': "Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at the current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnoses using all patient information and test results. Let's think step by step.\n### INPUT:\nA woman in her 80s presented to the emergency room (ER) with a 2-week history of enlarging left parietal scalp mass. She initially was seen by her primary care practitioner, where neurological examination was nonfocal. The mass continued to enlarge and she developed progressive generalized weakness, slowed speech, and focal right-handed weakness. Acute onset of headache and decline in mental status prompted presentation to the ER. On examination an 8-cm left parietal mass was tender to palpation. Treatment with 4 mg of dexamethasone was initiated intravenously. All biochemical laboratory results were normal. Brain magneti

In [9]:
print(ds['train']['entries'][0])
[{'content': "Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at the current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnoses using all patient information and test results. Let's think step by step.\n### INPUT:\nA woman in her 80s presented to the emergency room (ER) with a 2-week history of enlarging left parietal scalp mass. She initially was seen by her primary care practitioner, where neurological examination was nonfocal. The mass continued to enlarge and she developed progressive generalized weakness, slowed speech, and focal right-handed weakness. Acute onset of headache and decline in mental status prompted presentation to the ER. On examination an 8-cm left parietal mass was tender to palpation. Treatment with 4 mg of dexamethasone was initiated intravenously. All biochemical laboratory results were normal. Brain magnetic resonance imaging showed a large enhancing left frontal-parietal mass extending both extracranially and intracranially into the superior sagittal sinus (Figure). There was left frontal, parietal, and temporal bone involvement, and left dural thickening. Intracranially, there was adjacent vasogenic edema and associated mass effect. Computed tomographic (CT) scans of the chest, abdomen, and pelvis revealed a ground-glass sclerotic lesion of the right sacrum. Owing to involvement of the superior sagittal sinus and progressive growth, an ultrasound-guided core needle biopsy was performed.", 'role': 'human'}, {'content': '### Step 1:\nCutaneous abscess\n\n### Step 2:\nSquamous cell carcinoma of the scalp\n\n### Step 3:\nMeningioma\n\n### Step 4:\nMetastatic carcinoma\n\n### Step 5:\nNon-Hodgkin lymphoma\n\n### Step 6:\nMultiple myeloma\n\n### Step 7:\nPlasmacytoma\n\n### Step 8:\nGlioblastoma multiforme\n\n### Step 9:\nSkull osteomyelitis\n\n### Step 10:\nAngiosarcoma of the scalp\n\n---\n\n### Reranked Differential Diagnoses:\n\n1. Non-Hodgkin lymphoma\n2. Metastatic carcinoma\n3. Meningioma\n4. Multiple myeloma\n5. Plasmacytoma\n6. Angiosarcoma of the scalp\n7. Glioblastoma multiforme\n8. Squamous cell carcinoma of the scalp\n9. Skull osteomyelitis\n10. Cutaneous abscess', 'role': 'agent'}]

[{'content': "Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at the current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnoses using all patient information and test results. Let's think step by step.\n### INPUT:\nA 48-year-old man presented to the emergency department with a 1-week history of worsening right lower quadrant tenderness and fullness. This was associated with diarrhea, vomiting, fevers, and chills. He reported some relief of symptoms with loperamide. He denied any history of weight loss, melena, or hematochezia. Other than a previous open hemorrhoidectomy, the patient had no significant medical history. Of note, he had no family history of intestinal disease.\nOn examination, he had a fever with a temperature of 39°C and tachycardia with a heart rate of 102 beats per minute. On abdominal examination, there was localized tendernes

In [3]:
new_data = pd.read_csv('/home/gy237/project/llama3/new_data/not_selected_nejm_12.2/not_selected_nejm_generated_top10.tsv', sep = '\t')

In [4]:
print(new_data.loc[16, 'presentation_of_case'])

 A 48-year-old woman presented to the emergency department of this hospital with tingling of the right hand. One hour before presentation, the patient felt an acute “prickling and tingling” sensation diffusely on the right palm while she was reaching for a can of soda out of a vending machine. The sensation did not radiate to other areas of the hand or the arm, and it was not associated with decreased range of motion, tenderness, or joint pain. Although she presented to the emergency department 1 hour after the onset of symptoms, the tingling resolved before the examination, which took place approximately 80 minutes after presentation. The patient stated that she had recently traveled to Boston from New York City to visit museums, and she requested “accommodations” from medical personnel because she did not have a place to stay. On examination, the temperature was 36.9°C, the blood pressure 141/89 mm Hg, the heart rate 88 beats per minute, the respiratory rate 16 breaths per minute, an

In [35]:
prompt = '''Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at the current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnoses using all patient information and test results. Let's think step by step.\n### INPUT:\n'''

In [36]:
upload = []
for index, row in new_data.iterrows():
    question = prompt + row['refined_case_report']
    answer = row['output']
    upload.append({'entries': [{'content': question, 'role': 'human'}, {'content': answer, 'role': 'agent'}]})
print(len(upload))
for i in ds['train']:
    upload.append(i)
print(len(upload))

281
1396


In [37]:
train_dataset = Dataset.from_list(upload)
t = Dataset.from_list(upload[:10])

dataset_dict = DatasetDict({
    "train": train_dataset,
    "valid":t,
    "test": t
})
dataset_dict.push_to_hub("YBXL/NEJM-JAMA-reasoning-instruction")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/YBXL/NEJM-JAMA-reasoning-instruction/commit/d6383e19bde390ba3d2995a4f4ff2f5f84460d75', commit_message='Upload dataset', commit_description='', oid='d6383e19bde390ba3d2995a4f4ff2f5f84460d75', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
print(dataset_dict['train'][0])

{'entries': [{'content': 'Develop a 10-Step differential diagnosis, for each step, give only one differential diagnosis with only the disease name at the current step, do not provide repeated hypotheses or diagnoses that appear in previous steps. Rerank all 10 differential diagnoses using all patient information and test results. Let\'s think step by step.\n### INPUT:\nA 14-month-old boy was assessed at the pediatric gastroenterology clinic of this hospital due to episodes of vomiting. He was delivered via cesarean section at another hospital to a 35-year-old mother, following a 39-week and 6-day uncomplicated pregnancy, due to arrest in the second stage of labor. Meconium was present at birth. His Apgar scores were 9 at both 1 and 5 minutes, and he weighed 2.9 kg (23rd percentile) (Fig. 1). Breastfeeding commenced immediately. Newborn screening tests, including those for congenital diseases, hearing, and critical congenital heart disease, were normal, and he was discharged on the four

In [84]:
prompt = '''I will provide you with a case report and its corresponding final diagnosis. Your task is as follows:
Step 1: Determine whether the final diagnosis is associated with a gender-specific disease (e.g., uterine cancer). If the diagnosis is gender-specific, return: N. If it is not, proceed to Step 2.
Step 2: For non-gender-specific diseases, you should change the gender of the patient in the case report. Change the patient's gender (e.g., male to female or female to male, man to woman or woman to man) and adjust all associated pronouns accordingly. Ensure that no other information in the case report is altered; only the patient's gender and pronouns should be updated. Only respond the changed case report.'''

In [86]:
case_report = '''Final diagnosis: Pernicious anemia (vitamin B12 deficiency).
Case report:  A 48-year-old woman presented to the emergency department of this hospital with tingling of the right hand. One hour before presentation, the patient felt an acute “prickling and tingling” sensation diffusely on the right palm while she was reaching for a can of soda out of a vending machine. The sensation did not radiate to other areas of the hand or the arm, and it was not associated with decreased range of motion, tenderness, or joint pain. Although she presented to the emergency department 1 hour after the onset of symptoms, the tingling resolved before the examination, which took place approximately 80 minutes after presentation. The patient stated that she had recently traveled to Boston from New York City to visit museums, and she requested “accommodations” from medical personnel because she did not have a place to stay. On examination, the temperature was 36.9°C, the blood pressure 141/89 mm Hg, the heart rate 88 beats per minute, the respiratory rate 16 breaths per minute, and the oxygen saturation 99% while the patient was breathing ambient air. The weight was 70.9 kg, the height 160 cm, and the body-mass index (the weight in kilograms divided by the square of the height in meters) 27.7. The patient was described as having a labile affect, with rapid and tangential speech. The results of the remainder of the physical examination, including tests of neurologic function and examination of the muscles in the right hand and arm, were normal. The patient was discharged from the emergency department with a list of local shelters and hotels. Five hours after discharge, the patient felt recurrent tingling on the right palm, primarily along the radial-nerve distribution, and she again presented to the emergency department of this hospital. She reported that she had been unable to find a shelter and that she feared hotel staff would perform “unwanted sleep studies” or “gas” her. The results of the physical examination had not changed from several hours earlier. The patient was described as disheveled, unkempt, malodorous, and oriented to person and time. Examination revealed inconsistent thought processes and delusions of persecution. Urine toxicology screening was negative for amphetamines, barbiturates, benzodiazepines, cannabinoids, cocaine, opiates, and phencyclidine, and serum toxicology screening was negative for salicylates, acetaminophen, ethanol, and tricyclic antidepressants. A urine test for human chorionic gonadotropin was negative. Laboratory test results are shown in Table 1. Imaging studies were obtained. Dr. Brent P. Little: A radiograph of the right hand and wrist (Fig. 1) showed no acute fracture, dislocation, or soft-tissue abnormality; the joint spaces were preserved, and there were no joint erosions. Computed tomography (CT) of the head (Fig. 2), performed without the administration of intravenous contrast material, revealed no evidence of intracranial hemorrhage, mass, or acute territorial infarct. Dr. Crall: On evaluation by a psychiatrist, the patient incorrectly named the city and state as “Massatution” and “Boston,” respectively. Mental-status examination was notable for attentiveness to the interviewer, recall of two of five objects after 5 minutes, and the inability to subtract serial sevens and to repeat phrases. The patient’s score on the Montreal Cognitive Assessment was 21 (reference range, 26 to 30) on a scale of 0 to 30, with lower scores indicating greater cognitive impairment. She maintained eye contact and had no abnormal movements. Her speech was characterized by malapropisms and mispronunciations and was pressured. Her mood was described as happy, with a congruent affect. Her thought processes were described as tangential and disorganized. The results of the neurologic examination, including tests of cranial-nerve function, strength, and sensation to light touch, were normal. The patient reported no history of medical problems. She said that she had previously been a “volunteer client” at a psychiatric hospital in New York City but had not undergone psychiatric treatments or hospitalizations. A review of systems was negative for physical trauma, anorexia, weight loss, loss of energy, weakness, bleeding (menstrual cycles had stopped during the past year), headache, ataxia, dizziness, seizure, syncope, pain, depression, anhedonia, expansive mood, distractibility, anxiety, memory difficulties, sleep disturbance, auditory and visual hallucinations, suicidal ideation, and homicidal idea-tion. She took no medications and had no known allergies to medications. The patient reported the following information. She was born in the southeastern United States and grew up in an orphanage. She had skipped grades in elementary and high school because of high intelligence and then had obtained doctoral degrees in psychology from “Northwestern International University in Denmark” and “Panworld University in China.” Six years earlier, she had moved to New York City, where she lived in a downtown hotel. She was employed as a scientist to perform online research and volunteered for New York City hospitals and the U.S. Navy. She was unmarried and had no children, and she had no family or friends who could provide collateral information. She did not use alcohol, tobacco, or illicit drugs. Her family psychiatric and medical history was unknown, since she was adopted. She had no history of legal problems. Because of concern about the patient’s inability to protect herself in a new environment, given her impaired judgment and disorganized thinking, an order that authorized temporary involuntary hospitalization was implemented.1 The patient was admitted to the inpatient psychiatry unit. Results of urinalysis and the venous blood pH were normal; laboratory test results are shown in Table 1. Additional imaging studies were obtained. Dr. Little: A chest radiograph showed clear lungs. Magnetic resonance imaging (MRI) of the head (Fig. 2), performed without the administration of intravenous contrast material, revealed mild mucosal thickening in the paranasal sinus without acute intracranial abnormality. There was no evidence of infarct, such as intracranial mass, cerebral edema, or restricted diffusion, and there was no evidence of white-matter disease or volume loss. Dr. Crall: Laboratory test results that were obtained on the third hospital day are shown in Table 1. Results of additional diagnostic tests were received. '''

In [87]:
from openai import OpenAI
def generate(prompt, question):
    # return 'Y'    # use, when you check the diagnoses list
    client = OpenAI()
        
    chat_return = client.chat.completions.create(model='gpt-4o',temperature=0.0, messages=[{"role": "system", "content": prompt}, {"role":"user", "content": question}])

    result = chat_return.choices[0].message.content
    return result

In [88]:
answer = generate(prompt, case_report)
print(answer)

A 48-year-old man presented to the emergency department of this hospital with tingling of the right hand. One hour before presentation, the patient felt an acute "prickling and tingling" sensation diffusely on the right palm while he was reaching for a can of soda out of a vending machine. The sensation did not radiate to other areas of the hand or the arm, and it was not associated with decreased range of motion, tenderness, or joint pain. Although he presented to the emergency department 1 hour after the onset of symptoms, the tingling resolved before the examination, which took place approximately 80 minutes after presentation. The patient stated that he had recently traveled to Boston from New York City to visit museums, and he requested "accommodations" from medical personnel because he did not have a place to stay. On examination, the temperature was 36.9°C, the blood pressure 141/89 mm Hg, the heart rate 88 beats per minute, the respiratory rate 16 breaths per minute, and the oxygen saturation 99% while the patient was breathing ambient air. The weight was 70.9 kg, the height 160 cm, and the body-mass index (the weight in kilograms divided by the square of the height in meters) 27.7. The patient was described as having a labile affect, with rapid and tangential speech. The results of the remainder of the physical examination, including tests of neurologic function and examination of the muscles in the right hand and arm, were normal. The patient was discharged from the emergency department with a list of local shelters and hotels. Five hours after discharge, the patient felt recurrent tingling on the right palm, primarily along the radial-nerve distribution, and he again presented to the emergency department of this hospital. He reported that he had been unable to find a shelter and that he feared hotel staff would perform "unwanted sleep studies" or "gas" him. The results of the physical examination had not changed from several hours earlier. The patient was described as disheveled, unkempt, malodorous, and oriented to person and time. Examination revealed inconsistent thought processes and delusions of persecution. Urine toxicology screening was negative for amphetamines, barbiturates, benzodiazepines, cannabinoids, cocaine, opiates, and phencyclidine, and serum toxicology screening was negative for salicylates, acetaminophen, ethanol, and tricyclic antidepressants. A urine test for human chorionic gonadotropin was negative. Laboratory test results are shown in Table 1. Imaging studies were obtained. Dr. Brent P. Little: A radiograph of the right hand and wrist (Fig. 1) showed no acute fracture, dislocation, or soft-tissue abnormality; the joint spaces were preserved, and there were no joint erosions. Computed tomography (CT) of the head (Fig. 2), performed without the administration of intravenous contrast material, revealed no evidence of intracranial hemorrhage, mass, or acute territorial infarct. Dr. Crall: On evaluation by a psychiatrist, the patient incorrectly named the city and state as "Massatution" and "Boston," respectively. Mental-status examination was notable for attentiveness to the interviewer, recall of two of five objects after 5 minutes, and the inability to subtract serial sevens and to repeat phrases. The patient’s score on the Montreal Cognitive Assessment was 21 (reference range, 26 to 30) on a scale of 0 to 30, with lower scores indicating greater cognitive impairment. He maintained eye contact and had no abnormal movements. His speech was characterized by malapropisms and mispronunciations and was pressured. His mood was described as happy, with a congruent affect. His thought processes were described as tangential and disorganized. The results of the neurologic examination, including tests of cranial-nerve function, strength, and sensation to light touch, were normal. The patient reported no history of medical problems. He said that he had previously been a "volunteer client" at a psychiatric hospital in New York City but had not undergone psychiatric treatments or hospitalizations. A review of systems was negative for physical trauma, anorexia, weight loss, loss of energy, weakness, bleeding, headache, ataxia, dizziness, seizure, syncope, pain, depression, anhedonia, expansive mood, distractibility, anxiety, memory difficulties, sleep disturbance, auditory and visual hallucinations, suicidal ideation, and homicidal ideation. He took no medications and had no known allergies to medications. The patient reported the following information. He was born in the southeastern United States and grew up in an orphanage. He had skipped grades in elementary and high school because of high intelligence and then had obtained doctoral degrees in psychology from "Northwestern International University in Denmark" and "Panworld University in China." Six years earlier, he had moved to New York City, where he lived in a downtown hotel. He was employed as a scientist to perform online research and volunteered for New York City hospitals and the U.S. Navy. He was unmarried and had no children, and he had no family or friends who could provide collateral information. He did not use alcohol, tobacco, or illicit drugs. His family psychiatric and medical history was unknown, since he was adopted. He had no history of legal problems. Because of concern about the patient’s inability to protect himself in a new environment, given his impaired judgment and disorganized thinking, an order that authorized temporary involuntary hospitalization was implemented. The patient was admitted to the inpatient psychiatry unit. Results of urinalysis and the venous blood pH were normal; laboratory test results are shown in Table 1. Additional imaging studies were obtained. Dr. Little: A chest radiograph showed clear lungs. Magnetic resonance imaging (MRI) of the head (Fig. 2), performed without the administration of intravenous contrast material, revealed mild mucosal thickening in the paranasal sinus without acute intracranial abnormality. There was no evidence of infarct, such as intracranial mass, cerebral edema, or restricted diffusion, and there was no evidence of white-matter disease or volume loss. Dr. Crall: Laboratory test results that were obtained on the third hospital day are shown in Table 1. Results of additional diagnostic tests were received.


A 48-year-old man presented to the emergency department of this hospital with tingling of the right hand. One hour before presentation, the patient felt an acute "prickling and tingling" sensation diffusely on the right palm while he was reaching for a can of soda out of a vending machine. The sensation did not radiate to other areas of the hand or the arm, and it was not associated with decreased range of motion, tenderness, or joint pain. Although he presented to the emergency department 1 hour after the onset of symptoms, the tingling resolved before the examination, which took place approximately 80 minutes after presentation. The patient stated that he had recently traveled to Boston from New York City to visit museums, and he requested "accommodations" from medical personnel because he did not have a place to stay. On examination, the temperature was 36.9°C, the blood pressure 141/89 mm Hg, the heart rate 88 beats per minute, the respiratory rate 16 breaths per minute, and the ox